In [ ]:
// #r ".\binaries2\bossspad.dll"
// #r ".\binaries2\XNSEC.dll"
// #r "C:\BoSSS_Binaries\bossspad.dll"
// #r "C:\BoSSS_Binaries\XNSEC.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
//static var myDb = OpenOrCreateDatabase(@"C:\Databases\Things3");
// static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\default_bosss_db_comb_vel05_variablecp"); 
static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k10_cpVariableSelfMadecorrelation"); //
//static var myDb = OpenOrCreateDatabase(@"C:\Users\gutierrez\default_bosss_db_comb");
myDb.Sessions

Opening existing database '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation'.


Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


#0: { Error loading session }


In [ ]:
BoSSSshell.WorkflowMgm.Init("CounterFlowFlame2");
BoSSSshell.WorkflowMgm.Sessions


In [ ]:
myDb.Sessions[0].Export().Do()

Starting export process... 

Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


Data will be written to the following directory:


Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


Loading session c0b40ccd-a4cd-48f1-b6f7-c723ad701adf failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\gutierrez\CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation\sessions\c0b40ccd-a4cd-48f1-b6f7-c723ad701adf\Session.info'.'


C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\NO-PROJ__NO-NAME__c0b40ccd-a4cd-48f1-b6f7-c723ad701adf

In [ ]:
var myBatch = BoSSSshell.ExecutionQueues[3];
//MiniBatchProcessor.Server.StartIfNotRunning(true);
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:
int nCells = 12;
int dg = 5;
double velmultiplier = 0.2;

## Using hard coded control files for sending them to the hpccluster. this should be deleted later

first calculate MF

In [ ]:
// Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);
string ending        = "CFF_velmult02_dg2_k12_cpVariableSelfMadecorrelation";// "P" + dg + "K" + nCells+"VelMult"+velmultiplier+"RefinementCenterline_cpconst";
string jobName       = string.Format("FS_" + "CounterDiffFlame" + ending);
string StartupString = string.Format("cs:BoSSS.Application.XNSEC.FullNSEControlExamples.FS_CounterDiffusionFlame({0}, {1},{2},@\"{3}\")", dg, nCells,velmultiplier, myDb.Path);
var oneJob           = new Job(jobName, solver); 
oneJob.SetControlStatement(StartupString);
oneJob.NumberOfMPIProcs = 16; 
oneJob.NumberOfThreads = 1;
oneJob.Activate(myBatch);
//BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);
string ending        = "P" + dg + "K" + nCells;
string jobName       = string.Format("Full_" + "baseJobName" + ending + "cpVar");
string StartupString = string.Format("cs:BoSSS.Application.XNSEC.FullNSEControlExamples.Full_CounterDiffusionFlame({0}, {1},{2},@\"{3}\")", dg, nCells, 0.6, myDb.Path);
var oneJob           = new Job(jobName, solver); 
oneJob.SetControlStatement(StartupString);
oneJob.NumberOfMPIProcs = 8;
oneJob.NumberOfThreads = 1;
oneJob.Activate(myBatch);

Deploying job Full_baseJobNameP2K12 ... 


Deploying executables and additional files ...


Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\CounterFlowFlame2-XNSEC2021Jul20_175500


copied 52 files.


   copied 'amd64' runtime.


deployment finished.


## Postprocessing

In [ ]:
var sess = myDb.Sessions[0];
var timestep_FullChem = sess.Timesteps.Last();

In [ ]:
myDb

{ Session Count = 2; Grid Count = 4; Path = \\hpccluster\hpccluster-scratch\gutierrez\RefinementCenterline_cpconst_p3_morerefinement }

In [ ]:
double eps = 1e-3; 
double xleft  = 0+eps; 
double xright = 1.0-eps;
int nCells = 180;
double[]  _xNodes = GenericBlas.Linspace(xleft, xright, nCells + 1);
string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure", "MassFraction0","MassFraction1","MassFraction2","MassFraction3"}; 


In [ ]:
Dictionary<string, double[]> Results_FullComb = new Dictionary<string, double[]>();
Dictionary<string, double> ReferenceValuesDict = new Dictionary<string, double>();// Re-dimenzionalise variables

ReferenceValuesDict.Add("VelocityX", Convert.ToDouble(sess.KeysAndQueries["uRef"]));
ReferenceValuesDict.Add("VelocityY", Convert.ToDouble(sess.KeysAndQueries["uRef"]));
ReferenceValuesDict.Add("Temperature", Convert.ToDouble(sess.KeysAndQueries["TRef"]));
ReferenceValuesDict.Add("Pressure", Convert.ToDouble(sess.KeysAndQueries["pRef"]));
ReferenceValuesDict.Add("MassFraction0", 1.0);
ReferenceValuesDict.Add("MassFraction1", 1.0);
ReferenceValuesDict.Add("MassFraction2", 1.0);
ReferenceValuesDict.Add("MassFraction3", 1.0);
ReferenceValuesDict.Add("MassFraction4", 1.0);
ReferenceValuesDict.Add("Length", Convert.ToDouble(sess.KeysAndQueries["LRef"]));

Error: (4,55): error CS0103: The name 'sess' does not exist in the current context
(5,55): error CS0103: The name 'sess' does not exist in the current context
(6,57): error CS0103: The name 'sess' does not exist in the current context
(7,54): error CS0103: The name 'sess' does not exist in the current context
(13,52): error CS0103: The name 'sess' does not exist in the current context

In [ ]:
//Save results from the Full problem simulation
foreach(var varname in varnames){
    double[] Yvalues = new double[nCells+1];
    var field = timestep_FullChem.Fields.Where( Fi => (Fi.Identification == varname)).Last();
    for(int n = 0 ; n < _xNodes.Length; n++){
        double pickedVal = field.ProbeAt(new double[]{ _xNodes[n], 0.000001});    // Pick value along the central axis        
        Yvalues[n] = pickedVal* ReferenceValuesDict[varname];             // Re.dimenzionalise varliables and store it in array
    }
    Results_FullComb.Add(varname,Yvalues);
}

Error: (4,17): error CS0103: The name 'timestep_FullChem' does not exist in the current context
(7,33): error CS0103: The name 'ReferenceValuesDict' does not exist in the current context
(9,5): error CS0103: The name 'Results_FullComb' does not exist in the current context

In [ ]:
// Redimensionalise the x coordinate
double[] _xNodesDim = new double[_xNodes.Length];
 for(int n = 0 ; n < _xNodes.Length; n++){
     _xNodesDim[n] = _xNodes[n]*ReferenceValuesDict["Length"];
}


Error: (4,33): error CS0103: The name 'ReferenceValuesDict' does not exist in the current context

In [ ]:
 string path = System.IO.Path.ChangeExtension(@"C:\BoSSS\experimental\internal\src\private-gutz\L4-application\XNSEC2\Worksheets\Jupyter\CounterFlowFlameMatlabCode\BoSSS_data\", null);
string newPathName = path +"Full" + "XCoord" + ".txt";
using (StreamWriter s = new StreamWriter(newPathName)) {
    for (int i = 0; i < _xNodes.Length; i++) {
        s.Write(_xNodesDim[i].ToString("E16", System.Globalization.NumberFormatInfo.InvariantInfo) + " ");
    }
}

foreach (var varname in varnames) {
    string newPathName = path +"Full" + varname + ".txt";
    using (StreamWriter s = new StreamWriter(newPathName)) {
        double[] _yNodes = Results_FullComb[varname];
        for (int i = 0; i < _xNodes.Length; i++) { 
            s.Write(_yNodes[i].ToString("E16", System.Globalization.NumberFormatInfo.InvariantInfo) + " "); 
        }
    }
}

Error: (5,17): error CS0103: The name '_xNodesDim' does not exist in the current context
(12,28): error CS0103: The name 'Results_FullComb' does not exist in the current context

In [ ]:
static var myDb = OpenOrCreateDatabase(@"C:\Users\gutierrez\RefinementCenterline_cpconst_p3");

In [ ]:
myDb.Sessions

In [ ]:
myDb.Sessions[0].Export().Do()

Error: System.ArgumentOutOfRangeException: Index was out of range. Must be non-negative and less than the size of the collection. (Parameter 'index')
   at System.Collections.Generic.List`1.get_Item(Int32 index)
   at Submission#20.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)